# 🏭 Warehouse Location Problem with FlexQAOA

This notebook demonstrates solving a **Warehouse Location Problem** using Aqarios's FlexQAOA algorithm. The Warehouse Location Problem is a classic facility location optimization challenge that determines which warehouses to open and how to assign customers to minimize total cost. 🎯

The problem formulation considers:
- 🚚 Transportation costs from warehouses to customers
- 💰 Setup costs for opening new warehouses at given locations

## 🔍 Overview

1. **Setup**
2. **Definition of Warehouse Location Problem with Luna Model**
3. **Solving the optimization problem with Luna**
4. **Comparison of FlexQAOA and building blocks**
5. **Inspecting in Luna Dashboard**
6. **Q&A**

## 📦 Imports and Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from luna_quantum import Model, Variable, Vtype, quicksum
import os
from utils.plotting import solution_to_arrays
from luna_quantum.solve import algorithms
import getpass
from utils.plotting import plot_warehouse_network
from luna_quantum.solve.parameters.algorithms.base_params import (
    LinearQAOAParams,
    ScipyOptimizerParams,
)
from luna_quantum.solve.parameters.algorithms.quantum_gate.flex_qaoa import (
    IndicatorFunctionParams,
    OneHotParams,
    PipelineParams,
    QuadraticPenaltyParams,
)

if "LUNA_API_KEY" not in os.environ:
    # Prompt securely for the key if not already set
    os.environ["LUNA_API_KEY"] = getpass.getpass("Enter your Luna API key: ")

![LunaSolve Flow Algorithm Selection](plots/LunaSolve.png)

## 📋 Problem Data Definition

Here we define the key components of our warehouse location problem:


🏭 1. Warehouses (Potential Facilities)

In [ ]:
facilities = {
    "Munich": {"cost": 50, "capacity": 7},
    "Nuremberg": {"cost": 40, "capacity": 6},
}

🏥 2. Hospitals (Customers/Demand Points)

In [ ]:
hospitals = {
    "H2": {"location": "Munich-North", "demand": 4},
    "H3": {"location": "Nuremberg-South", "demand": 3},
    "H4": {"location": "Augsburg-West", "demand": 2},
    "H8": {"location": "Erlangen", "demand": 1},
}

🚛 3. Transportation Costs

In [ ]:
transport_costs = {
    ("H2", "Munich"): 0.15,
    ("H2", "Nuremberg"): 0.40,
    ("H2", "Augsburg"): 0.30,
    ("H2", "Regensburg"): 0.55,
    ("H3", "Munich"): 0.40,
    ("H3", "Nuremberg"): 0.12,
    ("H3", "Augsburg"): 0.50,
    ("H3", "Regensburg"): 0.35,
    ("H4", "Munich"): 0.45,
    ("H4", "Nuremberg"): 0.18,
    ("H4", "Augsburg"): 0.55,
    ("H4", "Regensburg"): 0.40,
    ("H8", "Munich"): 0.40,
    ("H8", "Nuremberg"): 0.15,
    ("H8", "Augsburg"): 0.45,
    ("H8", "Regensburg"): 0.30,
}

In [ ]:
plot_warehouse_network(facilities, hospitals, transport_costs)

# Modelling the Warehouse Location Problem with AqModels

In [ ]:
depot = "Munich"
m = Model("Warehouse Location Problem")

# add variables
x = {}
y = {}
with m.environment:
    for f in facilities:
        for h in hospitals:
            x[(f, h)] = Variable(vtype=Vtype.Binary, name=f"x_{f},{h}")
        y[f] = Variable(vtype=Vtype.Binary, name=f"y_{f}")

# add cost to transport goods
m.objective += quicksum(
    x[f, h] * transport_costs[(h, f)] * hospitals[h]["demand"]
    for f in facilities
    for h in hospitals
)

# add cost to open facility
m.objective += quicksum(y[f] * facilities[f]["cost"] for f in facilities)

# each hospital must be delivered
for h in hospitals:
    m.add_constraint(
        quicksum(x[f, h] for f in facilities) == 1, name=f"deliver_hospital_{h}"
    )

# at most usage of capacity
for f in facilities:
    m.add_constraint(
        quicksum(hospitals[h]["demand"] * x[f, h] for h in hospitals)
        <= facilities[f]["capacity"] * y[f],
        name=f"deliver_{f}",
    )

print(m)

# Solving the Warehouse Location Problem with LunaSolve

In [ ]:
scip = algorithms.SCIP()
job = scip.run(model=m, name="WLP with Scip")

In [ ]:
solution = job.result()
print(solution)

# 🔄 Model Transformation Pipeline (Upcoming Feature)

In [ ]:
from utils.transformation import move_constraints

t_model = move_constraints(m)
print(t_model)

In [ ]:
alg = algorithms.SimulatedAnnealing(num_reads=100)
job = alg.run(model=t_model, name="WLP with PA")

In [ ]:
print(job.result())

# 🔬 Indepth comparison of different FlexQAOA configurations
We will compare different FlexQAOA configurations with the same base configuration.

In [ ]:
shared_config = {
    "reps": 3,
    "initial_params": LinearQAOAParams(delta_beta=0.2, delta_gamma=0.2),
    "optimizer": ScipyOptimizerParams(method="BFGS"),
}

📊 Standard QAOA: A FlexQAOA pipeline without advanced constraint handling (effectively the default QAOA algorithm)

In [ ]:
qaoa = algorithms.FlexQAOA(
    **shared_config,
    pipeline=PipelineParams(
        indicator_function=None,
        one_hot=None,
        quadratic_penalty=QuadraticPenaltyParams(),
    ),
)

🔢 One-Hot Encoding FlexQAOA: A FlexQAOA with a one-hot encoding of the binary variables

In [ ]:
oh_qaoa = algorithms.FlexQAOA(
    **shared_config,
    pipeline=PipelineParams(
        indicator_function=None,
        one_hot=OneHotParams(),
        quadratic_penalty=QuadraticPenaltyParams(),
    ),
)

🚩 Indicator Function FlexQAOA: A FlexQAOA with an indicator function for the binary variables

In [ ]:
if_qaoa = algorithms.FlexQAOA(
    **shared_config,
    pipeline=PipelineParams(
        one_hot=None,
        indicator_function=IndicatorFunctionParams(),
        quadratic_penalty=QuadraticPenaltyParams(),
    ),
)

🎯 Full FlexQAOA: A FlexQAOA pipeline with full functionality enabled

In [ ]:
flex_qaoa = algorithms.FlexQAOA(**shared_config)

In [ ]:
oh_job = oh_qaoa.run(model=m, name="QAOA with Onehots")
if_job = if_qaoa.run(model=m, name="QAOA with Indicators Function")
flex_job = flex_qaoa.run(model=m, name="QAOA with FlexQAOA")
qaoa_job = qaoa.run(model=m, name="QAOA Default")

## ⚡ FlexQAOA Algorithm Configuration

**FlexQAOA** (Flexible Quantum Approximate Optimization Algorithm) is a quantum optimization algorithm that can handle constrained optimization problems efficiently! 🔧

### 🛠️ Algorithm Parameters:
- **reps**: Number of QAOA layers (depth of the quantum circuit) 🔄
- **initial_params**: Starting parameters for the optimization 🎯
- **optimizer**: Classical optimizer used to update QAOA parameters 🚀

In [ ]:
qaoa_sol = qaoa_job.result()
ohc_sol = oh_job.result()
if_sol = if_job.result()
flex_sol = flex_job.result()

## 📥 Solution Retrieval

The algorithm runs asynchronously on quantum hardware or simulators. The `job.result()` method retrieves the solution once the optimization is complete. ⏳

In [ ]:
import matplotlib.pyplot as plt

flex_obj, flex_count = solution_to_arrays(flex_sol)
ohc_obj, ohc_count = solution_to_arrays(ohc_sol)
if_obj, if_count = solution_to_arrays(if_sol)
qaoa_obj, qaoa_count = solution_to_arrays(qaoa_sol)
plt.figure(figsize=(12, 6))
plt.step(
    qaoa_obj, qaoa_count, label="Objective Function QAOA", color="#013CF2", linewidth=3
)
plt.step(
    flex_obj,
    flex_count,
    label="Objective Function FlexQAOA",
    color="#EBC25B",
    linewidth=3,
)
plt.step(
    ohc_obj,
    ohc_count,
    label="Objective Function XY-Mixer QAOA",
    color="#538A6A",
    linewidth=3,
)
plt.step(
    if_obj,
    if_count,
    label="Objective Function Indicator QAOA",
    color="#7F7F7F",
    linewidth=3,
)

# Add labels and title
plt.xlabel("Objective Value")
plt.ylabel("Percentage of Samples")
plt.title("Feasible solution")
plt.ylim(0, 1)

plt.grid(False)
plt.axvline(
    x=91.47, color="black", linestyle="--", linewidth=2, label="Optimal Solution"
)

# Position legend to the right outside the plot area
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")

# Adjust layout to prevent legend cutoff
plt.tight_layout()

# Show the plot
plt.show()

![LunaSolve Flow Algorithm Selection](plots/LunaDashboard.png)

## 🎉 Conclusion

This notebook demonstrated how to solve a **Warehouse Location Problem** using Luna Quantum's FlexQAOA algorithm! 🚀

### 🔑 Key Takeaways:

1. **Problem Modeling**: Easily model with Luna Model library, load existing models with translators or use our predefined use cases
2. **Quantum and Classical Algorithms**: Easily switch between algorithms by leveraging our intuitive interface and transformations  
3. **Solution Quality**: FlexQAOA applies the most efficient optimization pipeline to find better solutions faster

**Happy optimizing!** 🎯✨

![LunaSolve Flow Algorithm Selection](plots/ApplicationsAreas.png)